# 实验八：管道与重定向
___

## 实验目的与内容

### 实验目的

1. 掌握命令之间用管道连接的策略与方法
2. 了解标准输入、标准输出与标准错误等信息
3. 掌握重定向的方法与策略

### 实验内容

## 1. 管道（pipe）

管道是一种通讯机制，通常用于进程间的通讯（另一种进程间通信的机制是socket网络通讯），它表现出来的形式就是将前一个进程的输出(stdout)直接作为下一个进程的输入(stdin)。

这是一个例子：
```bash
ls -al /etc | less
```

这是另一个例子：

In [22]:
cat /etc/passwd | tail | cut -d: -f1,7

postfix:/sbin/nologin
sshd:/sbin/nologin
ntp:/sbin/nologin
tcpdump:/sbin/nologin
bio:/bin/bash
apache:/sbin/nologin
mysql:/sbin/nologin
dockerroot:/sbin/nologin
hadoop:/bin/bash
redis:/sbin/nologin


### 1.1 管道中的常用命令

#### `wc`：简单小巧的计数工具

wc命令用于统计并输出一个文件中行、单词和字节的数目：
```bash
ls -ld /etc/*/ | wc -l
```

#### `cut`：打印每一行的某个字段
```bash
cut /etc/passwd -d ':' -f 1,6
```

#### `grep`：在文本中或 stdin 中查找匹配字符串

该命令的一般形式为：
```bash
grep [命令选项] 用于匹配的表达式 [文件]
```

#### `sort`：排序命令

通常在查看命令后面结合管道使用：
```bash
cat /etc/passwd | sort -t':' -k 3 -n
```

#### `uniq`：去重命令

uniq命令只能去连续重复的行，不是全文去重，所以要达到预期效果，可以在去重之前先排序：
```bash
history | cut -c 8- | cut -d ' ' -f 1 | sort | uniq
```

## <font color="red">$\S$ 上机练习1</font>

说说下面这些命令的作用分别是什么。
```bash
history | cut -c 8- | cut -d ' ' -f 1 | sort | uniq

history|cut -c 8- |cut -d ' ' -f 1|sort|uniq -dc

history | cut -c 8- | cut -d ' ' -f 1 |sort |uniq -D
```

## 2. 重定向（redirection）

在此前的课程中你可能已经多次看到`>`和`>>`出现在命令行之中了，没错，这就是所谓的输出重定向。

一般来说，命令的输出一般是定向到`/dev/stdout`，输入一般为`/dev/stdin`，而命令的错误输出一般重定向到`/dev/stderr`。`/dev/stdin`一般指的是键盘等输入设备，而`/dev/stdout`和`/dev/stderr`一般都是定向到输出设备如显示屏终端等。

|  文件描述符 | 设备文件 | 说明 |
| --- | --- | --- |
| 0 | `/dev/stdin` | 标准输入文件 |
| 1 | `/dev/stdout` | 标准输出文件|
| 2 | `/dev/stderr` | 标准错误文件 |

### 2.1 简单重定向

- `> filename`是输出重定向到另一个文件当中，`>> filename`则是将输出追加到一个文件当中；
- 相应地，`< filename`则是输入重定向，即从文件中读取输入

### 2.2 heredoc重定向

heredoc重定向是一种比较特殊的方式，将多行内容输出到一个文件当中的方式：
```bash
cat > test.c <<EOF
#include <stdio.h>

int main()
{
    printf("hello world!");
    return 0;
}

EOF
```

这就是把`<<EOF`以及`EOF`之间的内容写入文件`test.c`。

> 注意不要混淆重定向与管道：
- 管道是默认连接前一个命令的输出到下一个命令的输入
- 重定向通常需要一个文件来建立两个命令的连接

### 2.3 标准错误重定向

比如在我们的目录下存在一个文件`test0.c`，但不存在另一个文件`test1.c`：

In [1]:
echo "hello world" > test0.c

那么，运行下面的命令同时会输出`test0.c`的内容，并输出`test1.c`不存在的错误信息：

In [2]:
cat test0.c test1.c

hello world
cat: test1.c: 没有那个文件或目录


如果我们将输出重定向到一个文件`test.log`：

In [3]:
cat test0.c test1.c > test.log

cat: test1.c: 没有那个文件或目录


错误仍然在，为什么？这是因为标准输出与标准错误虽然都输出到伪终端，但它们的文件描述符不同，并不是同一个东西。那么，如果我们需要将标准输出和标准错误都重定向到一个文件，怎么办呢？

In [4]:
cat test0.c test1.c > test.log 2>&1

这时候标准输出和标准错误都被重定向到`test.log`：

In [5]:
cat test.log

hello world
cat: test1.c: 没有那个文件或目录


还可以写成这种形式：

In [6]:
cat test0.c test1.c &>test.log

In [7]:
cat test.log

hello world
cat: test1.c: 没有那个文件或目录


### 2.4 多重重定向

我们可以使用`tee`命令，不但可以在标准输出上输出，还可以同时重定向到其他文件。

In [8]:
cat test0.c test1.c | tee test.log

cat: test1.c: 没有那个文件或目录
hello world


In [9]:
cat test.log

hello world


注意：`tee`只会重定向标准输出，而标准错误并没有重定向到文件。

### 2.5 创建和关闭文件描述符

我们这里只用到了0，1，2这三个文件描述符。事实上，还可以有3-8这些文件描述符可用，但需要我们去创建这些文件描述符。

首先，让我们看看都有哪些文件描述符在被使用：

In [11]:
ls -Al /dev/fd/

总用量 0
lrwx------ 1 bio bio 64 1月  11 14:27 0 -> /dev/pts/4
lrwx------ 1 bio bio 64 1月  11 14:27 1 -> /dev/pts/4
lrwx------ 1 bio bio 64 1月  11 14:27 2 -> /dev/pts/4
lr-x------ 1 bio bio 64 1月  11 14:27 3 -> /proc/19739/fd


那么，如果我们要创建文件描述符4，怎么做呢？我们可以用`exec`命令辅助创建文件描述符：

In [12]:
exec 4>myfile

In [13]:
ls -Al /dev/fd/

总用量 0
lrwx------ 1 bio bio 64 1月  11 14:28 0 -> /dev/pts/4
lrwx------ 1 bio bio 64 1月  11 14:28 1 -> /dev/pts/4
lrwx------ 1 bio bio 64 1月  11 14:28 2 -> /dev/pts/4
lr-x------ 1 bio bio 64 1月  11 14:28 3 -> /proc/19764/fd
l-wx------ 1 bio bio 64 1月  11 14:28 4 -> /home/bio/science/courses/bi028/linux/myfile


然后我们在执行命令的时候，就可以用重定向到这个文件描述符了：

In [15]:
cat test0.c test1.c 1>&4 2>&4

In [16]:
cat myfile

hello world
cat: test1.c: 没有那个文件或目录


如果需要关闭这个文件描述符，同样要用到`exec`命令：

In [17]:
exec 4>&-

In [18]:
ls -Al /dev/fd/

总用量 0
lrwx------ 1 bio bio 64 1月  11 14:32 0 -> /dev/pts/4
lrwx------ 1 bio bio 64 1月  11 14:32 1 -> /dev/pts/4
lrwx------ 1 bio bio 64 1月  11 14:32 2 -> /dev/pts/4
lr-x------ 1 bio bio 64 1月  11 14:32 3 -> /proc/19982/fd


### 2.6 使用黑洞文件完全屏蔽输出和错误

> 在类 UNIX 系统中，/dev/null，或称空设备，是一个特殊的设备文件，它通常被用于丢弃不需要的输出流，或作为用于输入流的空文件，这些操作通常由重定向完成。读取它则会立即得到一个EOF。

我们可以灵活运用这个文件，起到屏蔽输出或者错误的功能：

In [19]:
cat test0.c test1.c &>/dev/null

或者只要输出，不要错误：

In [20]:
cat test0.c test1.c 2>/dev/null

hello world


## <font color="red">$\S$上机练习2</font>

试着理解下面这段代码的作用，实际这段代码不会正常工作。请结合本章知识分析这段代码没有正确工作的原因，并设法解决这个问题。
```bash
while read filename; do
   echo $filename
done <<(ls)
```

<font color="red">提示</font>：想想管道与重定向的区别。